In [2]:
import sqlite3
import os
import pandas as pd

In [3]:
# Create connection 
db_connection = sqlite3.connect('air_quality_economic_data.db')
cursor = db_connection.cursor()

In [4]:
# Create Countries table
cursor.execute('''
    CREATE TABLE Countries (
        country_code TEXT PRIMARY KEY,
        country_name TEXT
    )
''')

# Create AirQuality table
cursor.execute('''
    CREATE TABLE AirQuality (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        country_code TEXT,
        city TEXT,
        datetime TEXT,
        parameter TEXT,
        units TEXT,
        value REAL
    )
''')

# Create EconomicIndicators table
cursor.execute('''
    CREATE TABLE EconomicIndicators (
        country_code TEXT,
        indicator_name TEXT,
        year INTEGER,
        value REAL,
        PRIMARY KEY (country_code, indicator_name, year),
        FOREIGN KEY (country_code) REFERENCES Countries (country_code)
    )
''')

# Create CountryAttributes table
db_connection.execute('''
    CREATE TABLE CountryAttributes (
        country_code TEXT PRIMARY KEY,
        attribute_name TEXT,
        value TEXT,
        FOREIGN KEY (country_code) REFERENCES Countries (country_code)
    )
''')

db_connection.commit()


In [14]:
# to map country code and country name, and alpha-3 code to alpha-2 code 
country_code_mapping = pd.read_csv('data/country_code.csv') 
alpha3_to_alpha2 = dict(zip(country_code_mapping['alpha-3'], country_code_mapping['alpha-2']))
country_code_to_name = dict(zip(country_code_mapping['alpha-2'], country_code_mapping['name']))

In [6]:
base_folder_path = 'data/openaq'

for country_folder in os.listdir(base_folder_path):
    country_path = os.path.join(base_folder_path, country_folder)
    if os.path.isdir(country_path):
        # use mapping to get country name
        country_code = country_folder 
        country_name = country_code_to_name.get(country_code, "Unknown")

        # check and insert country Countries
        cursor.execute("SELECT country_code FROM Countries WHERE country_code = ?", (country_code,))
        result = cursor.fetchone()
        if not result:
            cursor.execute("INSERT INTO Countries (country_code, country_name) VALUES (?, ?)", (country_code, country_name))
        
        for city_folder in os.listdir(country_path):
            city_path = os.path.join(country_path, city_folder)
            if os.path.isdir(city_path):
                for file in os.listdir(city_path):
                    if file.endswith('.csv'):
                        file_path = os.path.join(city_path, file)
                        data = pd.read_csv(file_path)

                        # delete useless columns
                        data.drop(columns=['location_id', 'sensors_id', 'location', 'lat', 'lon'], inplace=True, errors='ignore')

                        # Add country_code and city
                        data['country_code'] = country_code
                        data['city'] = city_folder

                        data.to_sql('AirQuality', db_connection, if_exists='append', index=False)


In [7]:
worldbank_folder = 'data/worldbank'  


for file in os.listdir(worldbank_folder):
    if file.endswith('.csv'):
        file_path = os.path.join(worldbank_folder, file)
        data = pd.read_csv(file_path)

        # map alpha3 to alpha2
        data['Country Code'] = data['Country Code'].map(alpha3_to_alpha2)

        # Handle different world bank data
        if '2021' in data.columns and '2022' in data.columns:
            data_long_format = data.melt(id_vars=["Country Code", "Country Name", "Indicator Name"],
                                         var_name="year",
                                         value_name="value")
            data_long_format.rename(columns={'Country Code': 'country_code', 'Indicator Name': 'indicator_name'}, inplace=True)
            data_long_format.drop(columns=['Country Name'], inplace=True)
            data_long_format.to_sql('EconomicIndicators', db_connection, if_exists='append', index=False)
        else:
            # Data do not contain time related data
            data.rename(columns={'Country Code': 'country_code', 'IncomeGroup': 'value'}, inplace=True)
            data['attribute_name'] = file.replace('.csv', '') 
            # Insert into CountryAttributes
            data[['country_code', 'attribute_name', 'value']].to_sql('CountryAttributes', db_connection, if_exists='append', index=False)


In [8]:
# number of records
tables = ['Countries', 'AirQuality', 'EconomicIndicators', 'CountryAttributes']
for table in tables:
    count_query = f"SELECT COUNT(*) FROM {table}"
    count = pd.read_sql_query(count_query, db_connection)
    print(f"Number of records in {table}: {count.iloc[0, 0]}")


Number of records in Countries: 57
Number of records in AirQuality: 41320964
Number of records in EconomicIndicators: 2660
Number of records in CountryAttributes: 265


In [9]:
# TEST
queries = [
    "PRAGMA table_info(Countries);",
    "PRAGMA table_info(AirQuality);",
    "PRAGMA table_info(EconomicIndicators);",
    "PRAGMA table_info(CountryAttributes);",
    "SELECT * FROM Countries LIMIT 5;",
    "SELECT * FROM AirQuality LIMIT 5;",
    "SELECT * FROM EconomicIndicators LIMIT 5;",
    "SELECT * FROM CountryAttributes LIMIT 5;"
]

for query in queries:
    result = db_connection.execute(query).fetchall()
    print(f"Query: {query}\nResult: {result}\n")

Query: PRAGMA table_info(Countries);
Result: [(0, 'country_code', 'TEXT', 0, None, 1), (1, 'country_name', 'TEXT', 0, None, 0)]

Query: PRAGMA table_info(AirQuality);
Result: [(0, 'id', 'INTEGER', 0, None, 1), (1, 'country_code', 'TEXT', 0, None, 0), (2, 'city', 'TEXT', 0, None, 0), (3, 'datetime', 'TEXT', 0, None, 0), (4, 'parameter', 'TEXT', 0, None, 0), (5, 'units', 'TEXT', 0, None, 0), (6, 'value', 'REAL', 0, None, 0)]

Query: PRAGMA table_info(EconomicIndicators);
Result: [(0, 'country_code', 'TEXT', 0, None, 1), (1, 'indicator_name', 'TEXT', 0, None, 2), (2, 'year', 'INTEGER', 0, None, 3), (3, 'value', 'REAL', 0, None, 0)]

Query: PRAGMA table_info(CountryAttributes);
Result: [(0, 'country_code', 'TEXT', 0, None, 1), (1, 'attribute_name', 'TEXT', 0, None, 0), (2, 'value', 'TEXT', 0, None, 0)]

Query: SELECT * FROM Countries LIMIT 5;
Result: [('SK', 'Slovakia'), ('SE', 'Sweden'), ('UZ', 'Uzbekistan'), ('PL', 'Poland'), ('PE', 'Peru')]

Query: SELECT * FROM AirQuality LIMIT 5;
Resu

In [15]:
query = """
    SELECT COUNT(DISTINCT city) AS total_cities
    FROM AirQuality;
"""
total_cities = pd.read_sql_query(query, db_connection)
print(total_cities)

   total_cities
0          1278


In [10]:
country_code = 'US'
query = """
    SELECT AVG(value) as average_pm25
    FROM AirQuality
    WHERE country_code = ?
    AND parameter = 'pm25'
    AND datetime BETWEEN '2021-01-01' AND '2021-12-31';
"""
average_pm25 = pd.read_sql_query(query, db_connection, params=(country_code,))
print(average_pm25)

   average_pm25
0      8.781307


In [11]:
query = """
    SELECT country_code, AVG(value) as average_pm10
    FROM AirQuality
    WHERE parameter = 'pm10'
    GROUP BY country_code
    ORDER BY average_pm10 DESC
    LIMIT 1;
"""
most_polluted_country = pd.read_sql_query(query, db_connection)
print(most_polluted_country)


  country_code  average_pm10
0           AE     81.330956


In [13]:
query = """
    SELECT a.country_code, AVG(a.value) as average_pm25, e.value as gdp
    FROM AirQuality a
    INNER JOIN EconomicIndicators e ON a.country_code = e.country_code
    WHERE a.parameter = 'pm25'
    AND e.indicator_name = 'GDP (current US$)'
    AND e.year = 2021
    GROUP BY a.country_code, e.value;
"""
country_pm25_gdp = pd.read_sql_query(query, db_connection)
print(country_pm25_gdp)


   country_code  average_pm25           gdp
0            AE    -89.357486  4.151788e+11
1            AT      9.329183  4.792954e+11
2            AU    -29.390119  1.559034e+12
3            BA     36.113353  2.364957e+10
4            BD     32.595926  4.162648e+11
5            BE      8.199656  6.007488e+11
6            CA      7.356550  2.001487e+12
7            CH      8.058817  8.134088e+11
8            CL     30.204219  3.165812e+11
9            CN     16.318743  1.782046e+13
10           CY     15.044725  2.948292e+10
11           CZ     13.169308  2.817912e+11
12           DE      7.629561  4.278504e+12
13           ES      9.811172  1.445652e+12
14           FI      4.360266  2.967767e+11
15           FR     10.216523  2.959356e+12
16           GB      8.048025  3.141506e+12
17           GR     19.795937  2.146678e+11
18           HR      8.067323  6.908124e+10
19           HU     14.844004  1.820900e+11
20           ID   -364.404629  1.186505e+12
21           IE     10.019564  5